In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
import joblib
import os

In [20]:
# Step 2: Load Dataset
def load_dataset(filepath):
    """
    Load a dataset from a CSV file.
    :param filepath: str, path to the dataset file
    :return: pd.DataFrame, loaded dataset
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"The file {filepath} does not exist.")
    return pd.read_csv(filepath)

In [21]:
def preprocess_data_without_filter(df, investment_value, price_per_kg, additional_cost=0):
    """
    Preprocess the input data, calculate total expenses, yield, and profitability.
    :param df: pd.DataFrame, dataset
    :param investment_value: float, amount invested
    :param price_per_kg: float, price of Gherkins per kg
    :param initial_cost: float, optional initial cost
    :return: pd.DataFrame, preprocessed dataset
    """
    df.drop('Total', axis=1, inplace=True)
    
    #remove column name unwanted spaces
    df = df.rename(columns=lambda x: x.strip())

    # Calculate total expenses
    df['Total_Expenses'] = df[['Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer', 'Fungicide', 'Insecticide', 'Others']].sum(axis=1)

    # Add investment and calculate yield, profit
    df['Investment'] = investment_value + additional_cost
    df['Estimated_Yield'] = (df['Investment'] / df['Total_Expenses']) * df['KG']
    df['Revenue'] = df['Estimated_Yield'] * price_per_kg
    df['Profit'] = df['Revenue'] - df['Investment']
    df['Is_Profitable'] = df['Profit'] > 0

    return df

In [22]:
import pandas as pd

def preprocess_data(df, investment_value, price_per_kg, additional_cost=0, area=None, acres=0):
    """
    Preprocess the input data, calculate total expenses, yield, and profitability.
    :param df: pd.DataFrame, dataset
    :param investment_value: float, amount invested
    :param price_per_kg: float, price of Gherkins per kg
    :param additional_cost: float, optional additional cost
    :param area: str, area filter (optional)
    :param acres: int, number of acres filter (optional)
    :return: pd.DataFrame, preprocessed dataset
    """
    df.drop('Total', axis=1, inplace=True)
    
    # Remove unwanted spaces from column names
    df = df.rename(columns=lambda x: x.strip())

    # Calculate total expenses
    df['Total_Expenses'] = df[['Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer', 'Fungicide', 'Insecticide', 'Others']].sum(axis=1)

    # Add investment and calculate yield, profit
    df['Investment'] = investment_value + additional_cost
    df['Estimated_Yield'] = (df['Investment'] / df['Total_Expenses']) * df['KG']
    df['Revenue'] = df['Estimated_Yield'] * price_per_kg
    df['Profit'] = df['Revenue'] - df['Investment']
    df['Is_Profitable'] = df['Profit'] > 0

    # Filter by area and acres if specified
    filtered_df = df.copy()
    if area is not None:
        filtered_df = filtered_df[filtered_df['Area'] == area]
    if acres > 0:
        filtered_df = filtered_df[filtered_df['Acre'] == acres]
        
    # If dataframe is empty, create new data frame with the specified area and calculate values based on acres
    if filtered_df.empty:
        filter_area = area
        filter_acres = 1
        
        # Filter by area and one acre
        refilter = df[(df['Area'] == filter_area) & (df['Acre'] == filter_acres)]
        
        if not refilter.empty:
            # Multiply all relevant columns by the given acres except investment value
            refilter.loc[:, 'Total_Expenses'] *= acres
            refilter.loc[:, 'Acre'] *= acres
            refilter.loc[:, 'KG'] *= acres
            refilter.loc[:, 'Estimated_Yield'] *= acres
            refilter.loc[:, 'Revenue'] *= acres
            refilter.loc[:, 'Profit'] *= acres
            refilter.loc[:, 'Land_Planting'] *= acres
            refilter.loc[:, 'Strate_Fertilizer'] *= acres
            refilter.loc[:, 'Liquid_Fertilizer'] *= acres
            refilter.loc[:, 'Fungicide'] *= acres
            refilter.loc[:, 'Insecticide'] *= acres
            refilter.loc[:, 'Others'] *= acres
            refilter.loc[:, 'Is_Profitable'] = refilter['Profit'] > 0
            
            filtered_df = refilter

    return filtered_df

In [23]:
# without filter preprocess


In [24]:
dataset_path = '../../data/Raw/Dataset.csv'
df = load_dataset(dataset_path)
filtered_df = load_dataset(dataset_path)
investment_value = 60000  # Example investment value
price_per_kg = 100  # Example price per KG
additional_cost = 5000  # Example initial cost
area = 'ampara'  # Example area to filter
acres = 2  # Example acres filter

In [25]:
# without filter
processed_df_without_filter = preprocess_data_without_filter(df, investment_value, price_per_kg, additional_cost)
if processed_df_without_filter is not None:    
    processed_df_without_filter.head()
else:
    print("Error: Preprocessing failed.")

In [26]:
processed_df_without_filter.head()

,Acre,Land_Planting,Strate_Fertilizer,Liquid_Fertilizer,Fungicide,Insecticide,Others,Area,KG,Total_Expenses,Investment,Estimated_Yield,Revenue,Profit,Is_Profitable
0,1,48000,156640,9872,47372,25538,9050,ampara,4971,296472,65000,1089.866834,108986.683397,43986.683397,True
1,1,44000,158900,9932,47600,21638,9950,bakamuna,2313,292020,65000,514.844874,51484.487364,-13515.512636,False
2,1,46000,157250,9870,49902,23638,9050,dehiaththakandiya,1793,295710,65000,394.119238,39411.923844,-25588.076156,False
3,1,47000,130200,11630,48030,24600,11000,girithale,4600,272460,65000,1097.408794,109740.879395,44740.879395,True
4,1,49200,145806,8950,49560,27650,9530,Kandakatiya,1238,290696,65000,276.818394,27681.839447,-37318.160553,False


In [27]:
processed_df = preprocess_data(filtered_df, investment_value, price_per_kg, additional_cost, area, acres)
if processed_df is not None:    
    processed_df.head()
else:
    print("Error: Preprocessing failed.")

In [28]:
processed_df.head()

,Acre,Land_Planting,Strate_Fertilizer,Liquid_Fertilizer,Fungicide,Insecticide,Others,Area,KG,Total_Expenses,Investment,Estimated_Yield,Revenue,Profit,Is_Profitable
301,2,84600,283600,20640,95000,49050,26220,ampara,9942,559110,65000,1155.819070,115581.906959,50581.906959,True
315,2,94834,330024,19547,94675,54681,22940,ampara,9300,616701,65000,980.215696,98021.569610,33021.569610,True
319,2,101418,304937,22287,101477,50456,17074,ampara,6026,597649,65000,655.384682,65538.468231,538.468231,True
320,2,108860,266602,20734,116943,44301,21037,ampara,7772,578477,65000,873.293147,87329.314735,22329.314735,True
322,2,94665,299572,28428,121129,44976,20490,ampara,4626,609260,65000,493.533139,49353.313856,-15646.686144,False


In [29]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score
import joblib
import pandas as pd

def preprocess_train_and_save_models(processed_df, processed_df_without_filter, regression_model_path, classification_model_path):
    """
    Preprocess the data, split it into features and target variables, train regression and classification models, and save the models.
    
    :param processed_df: pd.DataFrame, the processed dataframe with filtering applied
    :param processed_df_without_filter: pd.DataFrame, the processed dataframe without filtering applied
    :param regression_model_path: str, path to save the trained regression model
    :param classification_model_path: str, path to save the trained classification model
    :param expenses_model_path: str, path to save the trained expenses regression model
    :return: tuple, trained regression, classification, and expenses models
    """
    # Load your processed dataframe (without filtering) and drop rows with missing 'Area' values
    df = processed_df_without_filter.copy()
    df.dropna(subset=['Area'], inplace=True)
    processed_df.dropna(subset=['Area'], inplace=True)

    # Ensure 'Area' values are consistent (case, extra spaces) across both datasets
    df['Area'] = df['Area'].str.strip().str.lower()
    processed_df['Area'] = processed_df['Area'].str.strip().str.lower()

    # Check for unique 'Area' values in both datasets
    print("Original dataset 'Area' values:", df['Area'].unique())
    print("Processed dataset 'Area' values:", processed_df['Area'].unique())

    # Create the OneHotEncoder instance and fit it to the 'Area' column
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoder.fit(df[['Area']])

    # Transform the 'Area' column in the processed dataset
    encoded_area = encoder.transform(processed_df[['Area']])

    # Create a DataFrame with the encoded columns
    encoded_area_df = pd.DataFrame(encoded_area, columns=encoder.get_feature_names_out(['Area']))

    # Concatenate the original processed dataframe with the new encoded columns
    df_encoded = pd.concat([processed_df.reset_index(drop=True), encoded_area_df], axis=1)

    # Drop the original 'Area' column if no longer needed
    df_encoded.drop(columns=['Area'], inplace=True)

    # Display the updated DataFrame
    print("Processed Data with Encoded 'Area':")
    print(df_encoded.head())

    # Splitting Data for Regression, Classification, and Expenses Prediction
    X = df_encoded.drop(columns=['Profit', 'Is_Profitable', 'Total_Expenses'])
    y_regression = df_encoded['Profit']
    y_classification = df_encoded['Is_Profitable']
  

    # Split the data into training and testing sets
    X_train, X_test, y_reg_train, y_reg_test, y_class_train, y_class_test= train_test_split(
        X, y_regression, y_classification, test_size=0.2, random_state=42
    )

    # Train the regression model for Profit
    regression_model = LinearRegression()
    regression_model.fit(X_train, y_reg_train)
    y_reg_pred = regression_model.predict(X_test)
    reg_mse = mean_squared_error(y_reg_test, y_reg_pred)
    print(f"Regression Model Mean Squared Error: {reg_mse}")

    # Train the classification model
    classification_model = LogisticRegression()
    classification_model.fit(X_train, y_class_train)
    y_class_pred = classification_model.predict(X_test)
    class_accuracy = accuracy_score(y_class_test, y_class_pred)
    print(f"Classification Model Accuracy: {class_accuracy}")

    # Save the trained models to disk
    joblib.dump(regression_model, regression_model_path)
    joblib.dump(classification_model, classification_model_path)
    print(f"Regression model saved to {regression_model_path}")
    print(f"Classification model saved to {classification_model_path}")
    

    return regression_model, classification_model

In [30]:
# usage
regression_model_path = '../../models/regression_model.pkl'
classification_model_path = '../../models/classification_model.pkl'

regression_model, classification_model = preprocess_train_and_save_models(processed_df, processed_df_without_filter, regression_model_path, classification_model_path)

Original dataset 'Area' values: ['ampara' 'bakamuna' 'dehiaththakandiya' 'girithale' 'kandakatiya'
 'kanthale' 'mahiyanganaya' 'medirigiriya' 'monaragala' 'morawewa'
 'nikaweratiya' 'monagala' 'anganaya']
Processed dataset 'Area' values: ['ampara']
Processed Data with Encoded 'Area':
   Acre  Land_Planting  Strate_Fertilizer  Liquid_Fertilizer  Fungicide  \
0     2          84600             283600              20640      95000   
1     2          94834             330024              19547      94675   
2     2         101418             304937              22287     101477   
3     2         108860             266602              20734     116943   
4     2          94665             299572              28428     121129   

   Insecticide  Others    KG  Total_Expenses  Investment  ...  \
0        49050   26220  9942          559110       65000  ...   
1        54681   22940  9300          616701       65000  ...   
2        50456   17074  6026          597649       65000  ...   
3   

In [31]:
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def load_dataset(path):
    # Placeholder function to load dataset
    return pd.read_csv(path)

# Step 3: Preprocessing Function

def preprocess_data(df, investment_value, price_per_kg, additional_cost=0, area=None, acres=0):
    """
    Preprocess the input data, calculate total expenses, yield, and profitability.
    :param df: pd.DataFrame, dataset
    :param investment_value: float, amount invested
    :param price_per_kg: float, price of Gherkins per kg
    :param additional_cost: float, optional additional cost
    :param area: str, area filter (optional)
    :param acres: int, number of acres filter (optional)
    :return: pd.DataFrame, preprocessed dataset
    """
    df.drop('Total', axis=1, inplace=True)
    
    # Remove unwanted spaces from column names
    df = df.rename(columns=lambda x: x.strip())

    # Calculate total expenses
    df['Total_Expenses'] = df[['Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer', 'Fungicide', 'Insecticide', 'Others']].sum(axis=1)

    # Add investment and calculate yield, profit
    df['Investment'] = investment_value - additional_cost
    df['Estimated_Yield'] = (df['Investment'] / df['Total_Expenses']) * df['KG']
    df['Revenue'] = df['Estimated_Yield'] * price_per_kg
    df['Profit'] = df['Revenue'] - df['Investment']
    df['Is_Profitable'] = df['Profit'] > 0

    # Filter by area and acres if specified
    filtered_df = df.copy()
    if area is not None:
        filtered_df = filtered_df[filtered_df['Area'] == area]
    if acres > 0:
        filtered_df = filtered_df[filtered_df['Acre'] == acres]
        
    # If dataframe is empty, create new data frame with the specified area and calculate values based on acres
    if filtered_df.empty:
        filter_area = area
        filter_acres = 1
        
        # Filter by area and one acre
        refilter = df[(df['Area'] == filter_area) & (df['Acre'] == filter_acres)]
        
        if not refilter.empty:
            # Multiply all relevant columns by the given acres except investment value
            refilter.loc[:, 'Total_Expenses'] *= acres
            refilter.loc[:, 'Acre'] *= acres
            refilter.loc[:, 'KG'] *= acres
            refilter.loc[:, 'Estimated_Yield'] *= acres
            refilter.loc[:, 'Revenue'] *= acres
            refilter.loc[:, 'Profit'] *= acres
            refilter.loc[:, 'Land_Planting'] *= acres
            refilter.loc[:, 'Strate_Fertilizer'] *= acres
            refilter.loc[:, 'Liquid_Fertilizer'] *= acres
            refilter.loc[:, 'Fungicide'] *= acres
            refilter.loc[:, 'Insecticide'] *= acres
            refilter.loc[:, 'Others'] *= acres
            refilter.loc[:, 'Is_Profitable'] = refilter['Profit'] > 0
            
            filtered_df = refilter

    return filtered_df

def predict_profitability(investment_value, price_per_kg, initial_cost, area, acres, regression_model_path, classification_model_path):
    """
    Predict the profitability of Gherkin farming based on the input parameters.
    :param investment_value: float, amount invested
    :param price_per_kg: float, price of Gherkins per kg
    :param initial_cost: float, optional initial cost
    :param area: str, area filter (optional)
    :param acres: int, number of acres filter (optional)
    :param regression_model_path: str, path to the trained regression model
    :param classification_model_path: str, path to the trained classification model
    :param expenses_model_path: str, path to the trained expenses regression model
    :return: dict, predicted profitability
    """
    # Load the regression, classification, and expenses models
    regression_model = joblib.load(regression_model_path)
    classification_model = joblib.load(classification_model_path)    

    # Preprocess the input data
    dataset_path = '../../data/Raw/Dataset.csv'
    df = load_dataset(dataset_path)
    filtered_df = load_dataset(dataset_path)
    processed_df = preprocess_data(filtered_df, investment_value, price_per_kg, initial_cost, area, acres)

    # Strip column names of any leading or trailing spaces
    df.columns = df.columns.str.strip()
    processed_df.columns = processed_df.columns.str.strip()

    # Debug prints to check the DataFrame columns
    print("Initial columns in df:", df.columns)
    print("Initial columns in processed_df:", processed_df.columns)

    # Ensure 'Area' column exists before dropping missing values
    if 'Area' not in df.columns or 'Area' not in processed_df.columns:
        raise KeyError("The 'Area' column is missing from the DataFrame.")

    # Encode the 'Area' column
    df.dropna(subset=['Area'], inplace=True)
    processed_df.dropna(subset=['Area'], inplace=True)
    df['Area'] = df['Area'].str.strip().str.lower()
    processed_df['Area'] = processed_df['Area'].str.strip().str.lower()
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoder.fit(df[['Area']])
    encoded_area = encoder.transform(processed_df[['Area']])
    encoded_area_df = pd.DataFrame(encoded_area, columns=encoder.get_feature_names_out(['Area']))
    df_encoded = pd.concat([processed_df.reset_index(drop=True), encoded_area_df], axis=1)
    df_encoded.drop(columns=['Area'], inplace=True)

    # Debug prints to check the DataFrame columns after encoding
    print("Columns in df_encoded after encoding 'Area':", df_encoded.columns)

    # Predict profitability and expenses
    X = df_encoded.drop(columns=['Profit', 'Is_Profitable', 'Total_Expenses'])
    predicted_profit = regression_model.predict(X)[0]
    predicted_profitability = classification_model.predict(X)[0]    

    # Calculate adjusted profitability
    adjusted_profitability = predicted_profit - initial_cost
    is_profitable_adjusted = adjusted_profitability > investment_value

    # Return the predicted profitability
    return {
        'Predicted_Profit': predicted_profit,
        'Is_Profitable': bool(predicted_profitability),
        'Adjusted_Profitability': adjusted_profitability,
        'Is_Profitable_Adjusted': is_profitable_adjusted,
     
    }

In [38]:

# Example usage:
investment_value = 290000  # Example investment value
price_per_kg = 100  # Example price per KG
additional_cost = 300000  # Example initial cost
area = 'ampara'  # Example area to filter
acres = 1  # Example acres filter

predicted_profitability = predict_profitability(
    investment_value, price_per_kg, additional_cost, area, acres, '../../models/regression_model.pkl', '../../models/classification_model.pkl' 
)
# Determine Profit or Loss
profit_or_loss = "Profit" if predicted_profitability['Is_Profitable'] else "Loss"
adjusted_profit_or_loss = "Yes" if predicted_profitability['Is_Profitable_Adjusted'] else "No"

Initial columns in df: Index(['Acre', 'Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer',
       'Fungicide', 'Insecticide', 'Others', 'Total', 'Area', 'KG'],
      dtype='object')
Initial columns in processed_df: Index(['Acre', 'Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer',
       'Fungicide', 'Insecticide', 'Others', 'Area', 'KG', 'Total_Expenses',
       'Investment', 'Estimated_Yield', 'Revenue', 'Profit', 'Is_Profitable'],
      dtype='object')
Columns in df_encoded after encoding 'Area': Index(['Acre', 'Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer',
       'Fungicide', 'Insecticide', 'Others', 'KG', 'Total_Expenses',
       'Investment', 'Estimated_Yield', 'Revenue', 'Profit', 'Is_Profitable',
       'Area_ampara', 'Area_anganaya', 'Area_bakamuna',
       'Area_dehiaththakandiya', 'Area_girithale', 'Area_kandakatiya',
       'Area_kanthale', 'Area_mahiyanganaya', 'Area_medirigiriya',
       'Area_monagala', 'Area_monaragala', 'Area_morawewa',
       

In [41]:

# Display the results with real-world measurements

print(f"Total Cost: LKR {investment_value + additional_cost:.2f}")
print(f"Predicted Profit: LKR {predicted_profitability['Predicted_Profit']:.2f}")
print(f"Is Profitable (Model Prediction): {profit_or_loss}")
print(f"Total Income: LKR {investment_value + predicted_profitability['Predicted_Profit']:.2f}")
total_income = investment_value - additional_cost + predicted_profitability['Predicted_Profit']
difference_value = total_income - predicted_profitability['Predicted_Profit']

if difference_value == investment_value:
    print(f"No Defferece")
else:
    print(f"Difference: LKR {difference_value:.2f}")

Total Cost: LKR 590000.00
Predicted Profit: LKR -81767.18
Is Profitable (Model Prediction): Loss
Total Income: LKR 208232.82
Difference: LKR -10000.00


In [34]:
crac =  476507.73 - 186507.73
crac

290000.0

In [35]:
cc = load_dataset('../../data/Raw/Dataset.csv')
cc = cc.rename(columns=lambda x: x.strip())
cc['Area'].unique()


array(['ampara', 'bakamuna', 'dehiaththakandiya', 'girithale',
       'Kandakatiya', 'kanthale', 'mahiyanganaya', 'medirigiriya',
       'monaragala', 'morawewa', 'nikaweratiya', 'monagala ', 'monagala',
       'anganaya'], dtype=object)

In [37]:
import pandas as pd

# Define the areas and other parameters
areas = ['ampara', 'bakamuna', 'dehiaththakandiya', 'girithale',
         'Kandakatiya', 'kanthale', 'mahiyanganaya', 'medirigiriya',
         'monaragala', 'morawewa', 'nikaweratiya', 'monagala ', 'monagala',
         'anganaya']

investment_value = 290000  # Example investment value
price_per_kg = 100  # Example price per KG
additional_cost = 300000  # Example initial cost
acres = 1  # Example acres filter

# List to store the results
results = []

for area in areas:
    cc = load_dataset('../../data/Raw/Dataset.csv')
    cc = cc.rename(columns=lambda x: x.strip())
    filtered_df = preprocess_data(cc, investment_value, price_per_kg, additional_cost, area, acres)
    if filtered_df.empty:
        continue
    predicted_profitability = predict_profitability(
        investment_value, price_per_kg, additional_cost, area, acres, '../../models/regression_model.pkl', '../../models/classification_model.pkl'
    )
    profit_or_loss = "Profit" if predicted_profitability['Is_Profitable'] else "Loss"
    adjusted_profit_or_loss = "Yes" if predicted_profitability['Is_Profitable_Adjusted'] else "No"
    
    # Calculate total and difference
    total_income = investment_value + predicted_profitability['Predicted_Profit']
    difference = total_income - predicted_profitability['Predicted_Profit']
    
    # Append the results to the list
    results.append({
        'Area': area,
        'Total Cost': f"LKR {investment_value + additional_cost:.2f}",
        'Predicted Profit': f"LKR {predicted_profitability['Predicted_Profit']:.2f}",
        'Is Profitable (Model Prediction)': profit_or_loss,
        'Total Income': f"LKR {total_income:.2f}",
        'Difference': f"LKR {difference:.2f}"
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
results_df

Initial columns in df: Index(['Acre', 'Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer',
       'Fungicide', 'Insecticide', 'Others', 'Total', 'Area', 'KG'],
      dtype='object')
Initial columns in processed_df: Index(['Acre', 'Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer',
       'Fungicide', 'Insecticide', 'Others', 'Area', 'KG', 'Total_Expenses',
       'Investment', 'Estimated_Yield', 'Revenue', 'Profit', 'Is_Profitable'],
      dtype='object')
Columns in df_encoded after encoding 'Area': Index(['Acre', 'Land_Planting', 'Strate_Fertilizer', 'Liquid_Fertilizer',
       'Fungicide', 'Insecticide', 'Others', 'KG', 'Total_Expenses',
       'Investment', 'Estimated_Yield', 'Revenue', 'Profit', 'Is_Profitable',
       'Area_ampara', 'Area_anganaya', 'Area_bakamuna',
       'Area_dehiaththakandiya', 'Area_girithale', 'Area_kandakatiya',
       'Area_kanthale', 'Area_mahiyanganaya', 'Area_medirigiriya',
       'Area_monagala', 'Area_monaragala', 'Area_morawewa',
       

,Area,Total Cost,Predicted Profit,Is Profitable (Model Prediction),Total Income,Difference
0,ampara,LKR 590000.00,LKR -81767.18,Loss,LKR 208232.82,LKR 290000.00
1,bakamuna,LKR 590000.00,LKR -72920.69,Loss,LKR 217079.31,LKR 290000.00
2,dehiaththakandiya,LKR 590000.00,LKR -71063.37,Loss,LKR 218936.63,LKR 290000.00
3,girithale,LKR 590000.00,LKR -81883.21,Loss,LKR 208116.79,LKR 290000.00
4,Kandakatiya,LKR 590000.00,LKR -69258.74,Loss,LKR 220741.26,LKR 290000.00
5,kanthale,LKR 590000.00,LKR -76002.38,Loss,LKR 213997.62,LKR 290000.00
6,mahiyanganaya,LKR 590000.00,LKR -70962.59,Loss,LKR 219037.41,LKR 290000.00
7,medirigiriya,LKR 590000.00,LKR -87494.29,Loss,LKR 202505.71,LKR 290000.00
8,monaragala,LKR 590000.00,LKR -65700.45,Loss,LKR 224299.55,LKR 290000.00
9,morawewa,LKR 590000.00,LKR -68405.39,Loss,LKR 221594.61,LKR 290000.00
